In [1]:
import requests
import pandas as pd
import duckdb
import json
import datamart_profiler
id="datamart.zenodo.3733554.87ed9e35-0f51-49cb-a73c-c0e3581051f5"
filename=id.replace(".","_").replace("-","_")+".parquet"
resp=requests.get('https://auctus.vida-nyu.org/api/v1/download/'+id)

ModuleNotFoundError: No module named 'urllib3'

In [ ]:
data=resp.content.decode()
list=[x.split(',') for x in data.split('\n')]

df=pd.DataFrame(list[1:],columns=list[0])


In [2]:
df.columns

NameError: name 'df' is not defined

In [3]:
profiles=datamart_profiler.process_dataset(df)
for c in profiles['columns']:
    dtypes=[]
    dtypes.append(c['structural_type'].split('/')[-1])
    for s in c['semantic_types']:
        dtypes.append(s.split('/')[-1])
    print(c['name'],dtypes)

NameError: name 'datamart_profiler' is not defined

In [4]:
df.to_parquet(filename)
df.to_csv('test_file.csv')  

NameError: name 'df' is not defined

In [5]:
con=duckdb.connect("testdb.db")
con.sql(" describe select * from "+filename).fetchdf()

NameError: name 'duckdb' is not defined

In [6]:
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

NameError: name 'pd' is not defined

In [7]:
df_cpy=df.copy()
df_cpy
profiles=datamart_profiler.process_dataset(df_cpy)
for c in profiles['columns']:
    dtypes=[]
    dtypes.append(c['structural_type'].split('/')[-1])
    for s in c['semantic_types']:
        dtypes.append(s.split('/')[-1])
    try:
        if "DateTime" in dtypes:
            df_cpy[c['name']] = pd.to_datetime(df_cpy[c['name']], format='%Y-%m-%d')
        elif "Integer" in dtypes:
            df_cpy[c['name']]=df_cpy[c['name']].fillna(0).astype(float).astype(int)
        elif "Float" in dtypes:
            df_cpy[c['name']]=df_cpy[c['name']].fillna(0).astype(float)
        elif "Text" in dtypes:
            df_cpy[c['name']]=df_cpy[c['name']].fillna("null").astype(str)
    except:
        pass


NameError: name 'df' is not defined

In [8]:
df_cpy.dtypes

NameError: name 'df_cpy' is not defined

In [9]:
df_cpy.to_parquet("dummy_p.parquet")

NameError: name 'df_cpy' is not defined

In [10]:
con.sql(" describe select * from dummy_p.parquet ")

NameError: name 'con' is not defined

In [11]:
con.sql(" select date,count(*) from dummy_p.parquet group by date")

NameError: name 'con' is not defined

In [12]:
con.sql("select date,count(*) from dummy_p.parquet group by date having MONTH(date) between 1 and 2 ")

NameError: name 'con' is not defined

In [13]:
con.sql("select date from datamart_zenodo_3749559_e7fe094d_2f11_4814_ac1a_3326902e063c.parquet").to_df().to_json(orient='records')

NameError: name 'con' is not defined

In [14]:
x=con.sql("select date,count(*),month(date) from datamart_zenodo_3749559_e7fe094d_2f11_4814_ac1a_3326902e063c.parquet group by date")

NameError: name 'con' is not defined

In [48]:
x

┌─────────────────────┬──────────────┬─────────────┐
│        date         │ count_star() │ month(date) │
│      timestamp      │    int64     │    int64    │
├─────────────────────┼──────────────┼─────────────┤
│ 2020-03-24 00:00:00 │          195 │           3 │
│ 2020-03-25 00:00:00 │          199 │           3 │
│ 2020-03-26 00:00:00 │          200 │           3 │
│ 2020-03-27 00:00:00 │          201 │           3 │
│ 2020-03-28 00:00:00 │          201 │           3 │
│ 2020-03-29 00:00:00 │          201 │           3 │
│ 2020-03-30 00:00:00 │          202 │           3 │
│ 2020-03-31 00:00:00 │          204 │           3 │
│ 2020-04-01 00:00:00 │          205 │           4 │
│ 2020-04-02 00:00:00 │          206 │           4 │
│ 2020-04-03 00:00:00 │          207 │           4 │
│ 2020-04-04 00:00:00 │          208 │           4 │
│ 2020-04-05 00:00:00 │          210 │           4 │
│ 2020-04-06 00:00:00 │          211 │           4 │
│ 2020-04-07 00:00:00 │          211 │        

In [49]:
x.dtypes

['TIMESTAMP', 'BIGINT', 'BIGINT']